In [1]:
from sympy import *
from sympy.physics.mechanics import *

In [7]:
#initialize mechanics printing
mechanics_printing()

In [8]:
# define all variables

# generalized coordinates
q1, q2, q3, q4, q5, q6 = dynamicsymbols('q1:7')

In [14]:
# time
t = Symbol('t')

# masses
m1, m2, m3, m4, m5, m6 = symbols('m1:7')


# lengths
l1, l2, l3, l4, l5, l6 = symbols('l1:7')


In [15]:
# shortcut for derivatives:
q1d, q2d, q3d, q4d, q5d, q6d = dynamicsymbols('q1:7', 1)

In [19]:
#initialize fixed parameters
a0, a1, a2, a3, a4, a5 = symbols('a0:6')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5 = symbols('alpha0:6')
d1, d2, d3, d4, d5, d6 = symbols('d1:7')

## Forward Kinematics

The Position of the End-effector with respect to the base fram can be expressed as a multiplication of all of the transformation Matrices of the individual joints.

$$ _{0}^{EE}\textrm{T} = _{0}^{1}\textrm{T}_{1}^{2}\textrm{T} _{2}^{3} \textrm{T} _{3}^{4} \textrm{T} _{4}^{5} \textrm{T} _{5}^{6} \textrm{T} _{6}^{EE} \textrm{T} $$

Forunately the Denavit Hartenberg parameters are given by the  manufacturer of our example robot. So we can obtain the individual transformation Matrices by applying the DH parameters in the DH Transformation Matrix.


In [17]:
def dh_transformation_step(alpha, a, d, q):
    """
    returns a DH Transformation matrix.
    """
    transformation_matrix = Matrix([
        [cos(q), -sin(q), 0, a],
        [sin(q) * cos(alpha), cos(q) * cos(alpha), -sin(alpha), -sin(alpha) * d],
        [sin(q) * sin(alpha), cos(q) * sin(alpha), cos(alpha), cos(alpha) * d],
        [0, 0, 0, 1]
    ])
    return transformation_matrix

In [20]:
#define the denavit hartenberg parameters of the UR5
dh_params = {
    a0: 0, alpha0: 0,
    a1: 0, alpha1: pi/2, d1: 0.08916, q1: q1,
    a2: 0.425, alpha2: 0, d2: 0, q2: q2,
    a3: 0.39225, alpha3: 0, d3: 0, q3: q3,
    a4: 0, alpha4: pi/2, d4: 0.10915, q4: q4,
    a5: 0, alpha5: -pi/2, d5: 0.09456, q5: q5,
    d6: 0.0823, q6:q6
}

In [21]:
# calculate individual transformation matrices 
T01 = dh_transformation_step(alpha0, a0, d1, q1).subs(dh_params)
T12 = dh_transformation_step(alpha1, a1, d2, q2).subs(dh_params)
T23 = dh_transformation_step(alpha2, a2, d3, q3).subs(dh_params)
T34 = dh_transformation_step(alpha3, a3, d4, q4).subs(dh_params)
T45 = dh_transformation_step(alpha4, a4, d5, q5).subs(dh_params)
T56 = dh_transformation_step(alpha5, a5, d6, q6).subs(dh_params)

In [56]:
def get_positions(q1n, q2n, q3n, q4n, q5n, q6n):
    """takes radian values for each joints. Returns positions of each joint and End-Effector"""
    global T01, T12, T23, T34, T45, T56
    T01 = T01
    T02 = T01 * T12
    T03 = T02 * T23
    T04 = T03 * T34
    T05 = T04 * T45
    T06 = T05 * T56
    
    joint1 = T01.subs({q1: q1n, q2: q2n, q3: q3n, q4: q4n, q5: q5n, q6: q6n})[:, 3][:3]
    joint2 = T02.subs({q1: q1n, q2: q2n, q3: q3n, q4: q4n, q5: q5n, q6: q6n})[:, 3][:3]
    joint3 = T03.subs({q1: q1n, q2: q2n, q3: q3n, q4: q4n, q5: q5n, q6: q6n})[:, 3][:3]
    joint4 = T04.subs({q1: q1n, q2: q2n, q3: q3n, q4: q4n, q5: q5n, q6: q6n})[:, 3][:3]
    joint5 = T05.subs({q1: q1n, q2: q2n, q3: q3n, q4: q4n, q5: q5n, q6: q6n})[:, 3][:3]
    joint6 = T06.subs({q1: q1n, q2: q2n, q3: q3n, q4: q4n, q5: q5n, q6: q6n})[:, 3][:3]
    return joint1, joint2, joint3, joint4, joint5, joint6

In [57]:
j1, j2, j3, j4, j5, j6 = get_positions(0, 0, 0, 0, 0, 0)

In [58]:
j2

In [62]:
j6